In [ ]:
import json
from pprint import pprint
from statistics import mean, median, mode, stdev, variance, pstdev, pvariance
from copy import deepcopy, copy
from os import listdir, path, getcwd
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.collation import Collation

In [ ]:
base_uri = path.join(getcwd(), '..', 'hebrew_sources', 'sefaria-export', 'json', 'Tanakh')
tanakh = listdir(base_uri)
hebrew_bible = {}
english_bible = {}
for part in tanakh:
    directory_uri = path.join(base_uri, part)
    books = listdir(directory_uri)
    print(books)
    for book in books:
        book_uri = path.join(directory_uri, book)
        hebrew_uri = path.join(book_uri, 'Hebrew', 'Tanach with Nikkud.json')
        with open(hebrew_uri, 'r', encoding='utf-8') as file:
            data = json.load(file)
            hebrew_bible[book] = data['text']
        english_uri = path.join(book_uri, 'English', 'The Holy Scriptures A New Translation JPS 1917.json')
        with open(english_uri, 'r', encoding='utf-8') as file:
            data = json.load(file)
            english_bible[book] = data['text']

In [ ]:
bible_verses = []
bible_titles = hebrew_bible.keys()
for book_title in bible_titles:
    hebrew_book = hebrew_bible[book_title]
    english_book = english_bible[book_title]
    for chapter_num, chapter in enumerate(hebrew_book, start=1):
        for verse_num, verse in enumerate(chapter, start=1):
            verse_obj = {
                "book" : book_title,
                "chapter" : chapter_num,
                "verse" : verse_num,
                "hebrew" : hebrew_book[chapter_num - 1][verse_num - 1],
                "english" : english_book[chapter_num - 1][verse_num - 1]
            }
            bible_verses.append(verse_obj)

In [ ]:
bible_verses[-1]

In [ ]:
user = ""
password = ""
uri = ""

In [ ]:
conn_str = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(user, password, uri)
client = MongoClient(conn_str, server_api=ServerApi('1'))
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to server")

In [ ]:
databases = client.list_database_names()
print(f'These are the databases: {databases}')

In [ ]:
for verse in bible_verses:
    verse['chapter'] = str(verse['chapter'])
    verse['verse'] = str(verse['verse'])

In [ ]:
db = client.dictionaries
collection = db.bible_test

In [ ]:
collection.insert_many(bible_verses)